# Environment Setup
This needs to be run the first time you start the notebook

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [ ]:
#!connect mssql --kernel-name TPCCDemo "Persist Security Info=False; Integrated Security=true; Initial Catalog=tpcc_2000gb; Server=rp-sql19sl-01.perf.rubrik.com;"

In [ ]:
# Rubrik Connection Information
$Server = "10.8.49.101"
$ServiceAccountID = "User:::6acf798b-ffed-484a-a484-80846d5c6be2"
$Secret = "UI9TWr3uPflxjbr1D9ID8gvsmN6kgmygdDD0W9Og8lLi0a1vBPVZ3NvqVKjkaYwS+B2tlSYZF5tPvhzU/03X"

# Source Database Information
$SourceSQLServerInstance = "rp-sql19sl-01.perf.rubrik.com"
$SourceDatabaseName = "TPCC_2000GB"

# Target Database Information
$TargetSQLServerInstance = "rp-sql19sl-01.perf.rubrik.com"
$LiveMountName = "LiveMount"

# Look at the environment before

## List user databases and sizes

- <mark>Pay attention to the database name TPCC\_2000GB</mark>

In [ ]:
#!sql-TPCCDemo
with fs
as
(
    select database_id, type, size * 8.0 / 1024 size
    from sys.master_files
)
select
    name,
    (select sum(size) from fs where type = 0 and fs.database_id = db.database_id) DataFileSizeInMB,
    (select sum(size) from fs where type = 1 and fs.database_id = db.database_id) LogFileSizeInMB
from sys.databases db
where db.database_id > 4

## List the tables and their sizes

- <mark>Pay attention to the tables named district and warehouse</mark>
- Look at the row counts for both of these tables

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY 
    TotalSpaceMB DESC, t.Name

# Do a bad thing

## Delete without a WHERE clause

- A user is doing some work in the database. They need to do some clean up and they want to delete 5 entries from the warehouse table. However instead they made a mistake and left their WHERE clause commented out and mistakenly deleted all records from the table.

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
DELETE FROM warehouse
--WHERE W_id IN (1,2,3,4,5)

## Drop a table

- A Junior DBA is doing a change. They think they are in a non-production environment and mistakenly run this code against production.

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
DROP TABLE district

## List the tables and their sizes

- <mark>Pay attention to the tables named district and warehouse</mark>
- Notice that the warehouse table has 0 records
- Notice the district table no longer exists

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY t.Name

# Recover from the Bad Thing

## Live Mount the last backup

In [ ]:
# Connect-Rubrik with a Service Account
Connect-Rubrik -Server $Server -id $ServiceAccountID -Secret $Secret

# Get database information from Rubrik
$RubrikDatabase = Get-RubrikDatabase -Name $SourceDatabaseName -ServerInstance $SourceSQLServerInstance

#Mount a database to a SQL Server
$TargetInstance = Get-RubrikSQLInstance -ServerInstance $TargetSQLServerInstance
$RubrikRequest = New-RubrikDatabaseMount -id $RubrikDatabase.id `
	-TargetInstanceId $TargetInstance.id `
	-MountedDatabaseName $LiveMountName `
	-recoveryDateTime (Get-date (Get-RubrikDatabase -id $RubrikDatabase.id).latestRecoveryPoint) `
    -Confirm:$false
Get-RubrikRequest -id $RubrikRequest.id -Type mssql -WaitForCompletion
Disconnect-Rubrik

## Look at the environment after

### Look at the files of each database

In [ ]:
#!sql-TPCCDemo
USE [TPCC_2000GB]
SELECT
  name 'Logical Name', 
  physical_name 'File Location'
FROM sys.database_files;
USE [livemount]
SELECT
  name 'Logical Name', 
  physical_name 'File Location'
FROM sys.database_files;

### List user databases and sizes

In [ ]:
#!sql-TPCCDemo
with fs
as
(
    select database_id, type, size * 8.0 / 1024 size
    from sys.master_files
)
select
    name,
    (select sum(size) from fs where type = 0 and fs.database_id = db.database_id) DataFileSizeInMB,
    (select sum(size) from fs where type = 1 and fs.database_id = db.database_id) LogFileSizeInMB
from sys.databases db
where db.database_id > 4

## Recover the **warehouse** table

### Put records back

In [ ]:
#!sql-TPCCDemo
INSERT INTO TPCC_2000GB.dbo.warehouse
SELECT * FROM livemount.dbo.warehouse

## Recover the **district** table

### Create the district table

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[district](
	[d_id] [tinyint] NOT NULL,
	[d_w_id] [int] NOT NULL,
	[d_ytd] [money] NOT NULL,
	[d_next_o_id] [int] NULL,
	[d_tax] [smallmoney] NULL,
	[d_name] [char](10) NULL,
	[d_street_1] [char](20) NULL,
	[d_street_2] [char](20) NULL,
	[d_city] [char](20) NULL,
	[d_state] [char](2) NULL,
	[d_zip] [char](9) NULL,
	[padding] [char](6000) NOT NULL
) ON [PRIMARY]
GO
ALTER TABLE [dbo].[district] SET (LOCK_ESCALATION = DISABLE)
GO
ALTER TABLE [dbo].[district] ADD  CONSTRAINT [PK_DISTRICT] PRIMARY KEY CLUSTERED 
(
	[d_w_id] ASC,
	[d_id] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]
GO


### Put the records back

In [ ]:
#!sql-TPCCDemo
INSERT INTO TPCC_2000GB.dbo.district
SELECT * FROM livemount.dbo.district

## List the tables and their sizes

In [ ]:
#!sql-TPCCDemo
USE TPCC_2000GB
SELECT 
    t.NAME AS TableName,
    s.Name AS SchemaName,
    p.rows,
    SUM(a.total_pages) * 8 AS TotalSpaceKB, 
    CAST(ROUND(((SUM(a.total_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS TotalSpaceMB,
    SUM(a.used_pages) * 8 AS UsedSpaceKB, 
    CAST(ROUND(((SUM(a.used_pages) * 8) / 1024.00), 2) AS NUMERIC(36, 2)) AS UsedSpaceMB, 
    (SUM(a.total_pages) - SUM(a.used_pages)) * 8 AS UnusedSpaceKB,
    CAST(ROUND(((SUM(a.total_pages) - SUM(a.used_pages)) * 8) / 1024.00, 2) AS NUMERIC(36, 2)) AS UnusedSpaceMB
FROM 
    sys.tables t
INNER JOIN      
    sys.indexes i ON t.OBJECT_ID = i.object_id
INNER JOIN 
    sys.partitions p ON i.object_id = p.OBJECT_ID AND i.index_id = p.index_id
INNER JOIN 
    sys.allocation_units a ON p.partition_id = a.container_id
LEFT OUTER JOIN 
    sys.schemas s ON t.schema_id = s.schema_id
WHERE 
    t.NAME NOT LIKE 'dt%' 
    AND t.is_ms_shipped = 0
    AND i.OBJECT_ID > 255 
GROUP BY 
    t.Name, s.Name, p.Rows
ORDER BY t.Name

## Get Rid of the Live Mount

In [ ]:
# Connect-Rubrik with a Service Account
Connect-Rubrik -Server $Server -id $ServiceAccountID -Secret $Secret

# Unmount a database from SQL Server
$RubrikDatabaseMount = Get-RubrikDatabaseMount -MountedDatabaseName $LiveMountName -TargetInstanceId $TargetInstance.id
$RubrikRequest = Remove-RubrikDatabaseMount -id $RubrikDatabaseMount.id -Confirm:$false
Disconnect-Rubrik